In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
import chess
import numpy as np
import pandas as pd
import numexpr as ne
import swifter
import statsmodels.api as sm
import pylab

from nn import NeuralNetwork

In [28]:
df = pd.read_parquet("./random_evals_processed.parquet")
df.head(1)

,FEN,Evaluation,EvaluationStr,Prob,Bits,Hist
0,rnbqkb1r/pppppppp/B4n2/8/4P3/8/PPPP1PPP/RNBQK1...,-4.59,-459,0.015813,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0,...","[0, 0, 0, 0, 0]"


In [45]:
# df_filt = df[~((df['Evaluation'] == -318) | (df['Evaluation'] == 318))]
df_filt = df
mask = np.random.rand(len(df_filt)) < 0.8
train = df_filt[mask]
test = df_filt[~mask]

In [ ]:

    # PREDICTING EVALUATION


In [255]:
nn = NeuralNetwork([
    Layer(389, 1, 'identity')
])

In [299]:
nn.train(
    df = train, in_col = 'Bits', out_col = 'Evaluation',
    batch_size = 200_000,
    alpha = 0.5,
    stopper = lambda err, i : i == 10000
)

perror: [[4922.44711762]]
ferror: [[4918.99760561]]
perror: [[4869.91707602]]
ferror: [[4866.63180452]]
perror: [[4932.56032996]]
ferror: [[4929.39376609]]
perror: [[4862.46418051]]
ferror: [[4859.44279057]]
perror: [[4846.83325687]]
ferror: [[4843.88886164]]
perror: [[4940.97717044]]
ferror: [[4937.84964729]]
perror: [[4891.44695031]]
ferror: [[4888.40561464]]
perror: [[4840.73053616]]
ferror: [[4837.79608822]]
perror: [[4850.40896588]]
ferror: [[4847.47239491]]
perror: [[4892.17833932]]
ferror: [[4889.21502648]]
perror: [[4881.38562234]]
ferror: [[4878.58512486]]
perror: [[4868.12052658]]
ferror: [[4865.22151217]]
perror: [[4845.2717347]]
ferror: [[4842.44213093]]
perror: [[4869.65132752]]
ferror: [[4866.72469688]]
perror: [[4800.51852084]]
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\mehra\.julia\conda\3\lib\site-packages\swifter\swifter.py", line 311, in apply
    tmp_df = func(sample, *args, **kwds)
  File "C:\Users\mehra\Documents\GitHub\nnue-from-scratch\nn.py", line 12, in predict
    x = layer.predict(x) # pipe result to next layer
  File "C:\Users\mehra\AppData\Local\Temp\ipykernel_11152\1779428570.py", line 14, in predict
    x = np.array(x).reshape(self.inputs, 1)
ValueError: cannot reshape array of size 1000 into shape (389,1)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mehra\.julia\conda\3\lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mehra\AppData\Local\Temp\ipykernel_11152\2198872102.py", line 1, in <module>
    nn.train(
  File "C:\Users\mehra\Documents\GitHub\nnue-from-scratch\nn.py", line 103, in train
    err = self.batch

In [370]:
nn.model_stats(test['Bits'], test['Evaluation'], 0)

AVERAGE ABSOLUTE ERROR: 28.32387356556481
AVERAGE SQUARE ERROR:   4.137776491563141
SIGN ACCURACY:          0.7183230618879515


In [337]:
def getWeights(pc):
    if pc == 0:
        return nn.layers[0].A[0][8: 64 - 8]
    return nn.layers[0].A[0][pc * 64: (pc + 1) * 64]

def printWeights(pc):
    w = getWeights(pc)
    for i in range(len(w)):
        print(w[i])
        if i % 8 == 7:
            print()

def getVal(pc):
    return np.average(getWeights(pc))

In [326]:
getVal(4)

14.796467169233841

In [327]:
getWeights(0)

array([10.42160825,  7.59611841,  7.11966815,  4.06069137,  7.15220186,
        7.82908104, 12.4693455 , 12.62915352,  8.72859756,  4.53348943,
        5.44165368,  6.67777387,  9.83919608,  6.99972737, 13.31568342,
       14.96311622,  8.88045066,  6.12880273,  7.39608471,  5.20646709,
        7.77688619,  6.34220503,  6.42644058, 10.29728061, -0.87715629,
        6.9832149 ,  9.64129104,  1.93299495,  9.37861178,  8.82834754,
       11.86392237, 10.77456812,  9.2446435 , 18.94605444,  5.11876588,
       11.47165346, 19.28105496, 25.19016491, 16.37526205, 15.92733023,
       17.13170263, 22.62119041, 22.83549716, 20.18726589, 28.0574901 ,
       12.10628516,  4.588379  , 11.12057327])

In [333]:

    # PREDICTING WIN PROBABILITY


In [334]:
nn2 = NeuralNetwork([
    Layer(389, 1, 'identity'),
])

In [346]:
nn2.train(
    df = train, in_col = 'Bits', out_col = 'Prob',
    batch_size = 512,
    alpha = 0.01,
    stopper = lambda err, i : i == 10000
)

perror: [[0.10718162]]
ferror: [[0.10704229]]
perror: [[0.08051619]]
ferror: [[0.08042472]]
perror: [[0.10482782]]
ferror: [[0.10462098]]
perror: [[0.09524207]]
ferror: [[0.09508803]]
perror: [[0.09886314]]
ferror: [[0.0987326]]
perror: [[0.09103487]]
ferror: [[0.09096134]]
perror: [[0.08917481]]
ferror: [[0.08902522]]
perror: [[0.09530253]]
ferror: [[0.09514314]]
perror: [[0.10263644]]
ferror: [[0.10247689]]
perror: [[0.08979405]]
ferror: [[0.08970385]]
perror: [[0.09254477]]
ferror: [[0.09243134]]
perror: [[0.09253555]]
ferror: [[0.09240915]]
perror: [[0.08248674]]
ferror: [[0.08237879]]
perror: [[0.08745213]]
ferror: [[0.08735306]]
perror: [[0.08618443]]
ferror: [[0.08606926]]
perror: [[0.10681593]]
ferror: [[0.10661297]]
perror: [[0.09253581]]
ferror: [[0.09239728]]
perror: [[0.08954982]]
ferror: [[0.0894438]]
perror: [[0.09228442]]
ferror: [[0.0921377]]
perror: [[0.10680557]]
ferror: [[0.10662444]]
perror: [[0.0888451]]
ferror: [[0.08873471]]
perror: [[0.09012325]]
ferror: [[0.090

KeyboardInterrupt: 

In [367]:
nn2.model_stats(test['Bits'], test['Prob'], 0.5)

AVERAGE ABSOLUTE ERROR: 0.2463541154766824
AVERAGE SQUARE ERROR:   0.04074644372108502
SIGN ACCURACY:          0.7966951873790862


In [1226]:

    # COMPARE W/ SKLEARN


In [1537]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(2, 1), random_state=1)

In [ ]:
ins = [[float(x) for x in ls] for ls in ins]
outs = [float(x) for x in outs]

In [1556]:
X = [[0., 0.], [1., 1.]]
y = [0, 1]
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X, y)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')

In [ ]:

    # MNIST


In [18]:
mdf = pd.read_csv('./mnist_test.csv')

In [19]:
mdf.head(5)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
def cols_to_img(row):
    return np.array(row)[1:(28*28 + 1)]

In [21]:
mdf['img'] = mdf.swifter.apply(cols_to_img, axis=1)

Pandas Apply:   0%|          | 0/10000 [00:00<?, ?it/s]

In [22]:
def label_to_vec(label):
    arr = np.zeros(10, dtype=float)
    arr[int(label)] = 1.
    return arr

In [23]:
mdf['out'] = mdf['label'].apply(label_to_vec)

In [90]:
mdf['out'] = mdf['out'].apply(lambda x : np.array([x]).T)
mdf['img'] = mdf['img'].apply(lambda x : np.array([x]).T)

In [537]:
mnn = NeuralNetwork([
    Layer(784, 10, 'sigmoid')
])

In [538]:
mnn.backprop(mdf['img'][0], mdf['out'][0])[0][1].shape

(10, 784)

In [539]:
mnn.predict(mdf['img'][0])

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [540]:
mnn.train(
    df = mdf, in_col = 'img', out_col = 'out',
    batch_size = 100,
    alpha = 0.0001
)

perror: [[1.94]
 [1.83]
 [1.89]
 [1.95]
 [1.86]
 [1.93]
 [1.87]
 [1.94]
 [1.88]
 [1.91]]
ferror: [[1.94]
 [1.83]
 [1.89]
 [1.95]
 [1.86]
 [1.93]
 [1.87]
 [1.94]
 [1.88]
 [1.91]]
perror: [[1.89]
 [1.88]
 [1.89]
 [1.89]
 [1.93]
 [1.87]
 [1.92]
 [1.89]
 [1.92]
 [1.92]]
ferror: [[1.89]
 [1.88]
 [1.89]
 [1.89]
 [1.93]
 [1.87]
 [1.92]
 [1.89]
 [1.92]
 [1.92]]
perror: [[1.9 ]
 [1.91]
 [1.96]
 [1.81]
 [1.89]
 [1.89]
 [1.92]
 [1.89]
 [1.95]
 [1.88]]
ferror: [[1.9 ]
 [1.91]
 [1.96]
 [1.81]
 [1.89]
 [1.89]
 [1.92]
 [1.89]
 [1.95]
 [1.88]]
perror: [[1.91]
 [1.84]
 [1.88]
 [1.92]
 [1.94]
 [1.9 ]
 [1.92]
 [1.88]
 [1.91]
 [1.9 ]]
ferror: [[1.91]
 [1.84]
 [1.88]
 [1.92]
 [1.94]
 [1.9 ]
 [1.92]
 [1.88]
 [1.91]
 [1.9 ]]
perror: [[1.94]
 [1.85]
 [1.88]
 [1.91]
 [1.91]
 [1.96]
 [1.92]
 [1.82]
 [1.84]
 [1.97]]
ferror: [[1.94]
 [1.85]
 [1.88]
 [1.91]
 [1.91]
 [1.96]
 [1.92]
 [1.82]
 [1.84]
 [1.97]]
perror: [[1.9 ]
 [1.95]
 [1.94]
 [1.86]
 [1.85]
 [1.95]
 [1.89]
 [1.94]
 [1.81]
 [1.91]]
ferror: [[1.9 ]
 [1.9

KeyboardInterrupt: 

In [ ]:
mnn.backprop()